# Visualização de dados  <br />

#### Objetivo:

In [7]:
import pandas as pd 
import import_ipynb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import kde

###### Classe 1:  Verificando_dados:
Para verificar valores que estão faltando  e valores duplicados

#### Classe 2: Analise_exploratoria:
Funções:  
   * Mapa de correlação 
   * Distribuição de valores das colunas.
   * sns_lmplot

In [8]:
class Verificando_dados:
    
    def missing_data(self, data):
        total = data.isnull().sum()

        percent = (data.isnull().sum()/data.isnull().count()*100)
        tt = pd.concat([total,percent], axis=1, keys = ['Total', 'Percent'])
        types =[]
        for col in data.columns:
            dtype = str(data[col].dtype)
            types.append(dtype)
        tt['Types'] = types
        self.mostrar_faltantes =  tt.loc[(tt['Total'] != 0 )]
        
    
    def Valores_duplicados(self, data):

        features = data.columns.values[0:]

        unique_max_data = []
        
        for feature in features:
            values = data[feature].value_counts()
            unique_max_data.append([feature, values.max(), values.idxmax()])
        
        self.mostrar_duplicados = np.transpose(pd.DataFrame(unique_max_data, columns=['Feature', 'Max duplicates', 'Value']).sort_values(by='Max duplicates', ascending=False))
        

In [25]:
class Analise_exploratoria:
    
    def mapa_de_correlacao(self, dados_pre_processados_EAD, método):
         
        corr_df1 = dados_pre_processados_EAD.corr(method=método)

        # Cria um  mask para mostrar apenas a parte inferior do  triângulo 
        mask = np.zeros_like(corr_df1)
        mask[np.triu_indices_from(mask)] = True
        # Cria uma tabela de correlação usando  seaborn. 
        # List if colormaps (parameter 'cmap') Disponivel aqui: http://matplotlib.org/examples/color/colormaps_reference.html
        sns.heatmap(corr_df1, cmap='RdYlGn_r', vmax=1.0, vmin=-1.0 , mask = mask, linewidths=2.5)

        # Para orientar as labels.
        plt.yticks(rotation=0) 
        plt.xticks(rotation=90) 
        plt.show()
        
    def distribuicao_dos_valores(self, _dados_, coluna ):
       
        
        t0 =_dados_[_dados_[coluna]==0]
        t1 =_dados_[_dados_[coluna]==1]


        print('Distribuição dos valores das colunas')
        plt.figure(figsize=(20,16))
        for i, col in enumerate(list(_dados_.columns)[2:30]): 
            plt.subplot(7,4,i+1)
            plt.hist(t0[col],label='targer 0', color = 'blue')
            plt.hist(t1[col],label='targer 1', color = 'magenta')

            plt.title (col)
            plt.grid()
            plt.legend(loc='upper right')
            plt.tight_layout()
            
    def sns_jointplot(self, var_1,var_2,  df, hue):
      
 #         # plt.title( var_2 +  " Vs " +  var_1 )
 
          a = sns.jointplot(x= var_1, y= var_2, data=df,  hue=hue, palette='Set1',  kind="kde")#,ax=ax)
          plt.gcf().set_size_inches(7, 7)
        
                

        
    def scatterplot_with_2d_density(self, dados_pre_processados_EAD, eixo_x, eixo_y ):
        #https://python-graph-gallery.com/86-avoid-overlapping-in-scatterplot-with-2d-density/
        paran=  [eixo_x, eixo_y]
        data = dados_pre_processados_EAD[paran]
        data = np.array(data)
        x, y = data.T

        # Create a figure with a plot area
        fig, axes = plt.subplots(ncols=1, nrows=1, figsize=(6.6, 5))
        
        nbins = 20

        # Evaluate a gaussian kde on a regular grid of nbins x nbins over data extents

        k = kde.gaussian_kde(data.T)
        xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
        zi = k(np.vstack([xi.flatten(), yi.flatten()]))
 

        # Contour

        axes.set_title(eixo_y +  " Vs " +  eixo_x)
        axes.set_xlabel(eixo_x)
        axes.set_ylabel(eixo_y)


        axes.pcolormesh(xi, yi, zi.reshape(xi.shape), shading='gouraud', cmap=plt.cm.seismic)
        axes.contour(xi, yi, zi.reshape(xi.shape) )    

    def grafico_violino(self, dados_pre_processados_EAD, eixo_y):
        plt.subplots(figsize=(10, 5))
        x = list(dados_pre_processados_EAD[eixo_y].columns)
        for i, col in enumerate(x):
            plt.subplot(2,2,i+1)
            axes = sns.violinplot(x='salary', y = col , data=dados_pre_processados_EAD)
            #plt.show()
            plt.tight_layout()

    def histograma(self,data ):
        
        dados_pre_processados_EAD = data
        fig = plt.figure( figsize = (16, 40))

        #import matplotlib.pyplot as plt


        # Dados de fora 


        colors  = ['b','y']
        label_x = "Não Sairam"
        label_y = "Sairam"



        dado_0_  =  dados_pre_processados_EAD.loc[(dados_pre_processados_EAD["left_Company"] != 0)]
        dado_1_  =  dados_pre_processados_EAD.loc[(dados_pre_processados_EAD["left_Company"] != 1)]    

        new_features=[]

        def Funcao_auxiliar(col): 
            variavel   = [dado_0_[col], dado_1_[col]]
            labels_0   = list(set(variavel[0]))
            labels_1   = list(set(variavel[1]))
            
            size_0     = len(labels_0)
            size_1     = len(labels_1)
           
            size = max(size_0, size_1)

            if size_0 > size_1:
                labels = labels_0
            else:
                labels = labels_1


            return variavel, labels, size

        for  col  in list(dados_pre_processados_EAD.columns): 
            variavel, labels, size = Funcao_auxiliar(col)
            if  size <= 20:
                new_features.append(col)

        for i, col  in enumerate(dados_pre_processados_EAD.columns):  

             # parâmetros do histograma
            variavel, labels, size = Funcao_auxiliar(col)


            #Definindo a altura do eixo  Y - Define o número máximo de pontos no eixo y
            max_eixo_y=  max(max(variavel[0].value_counts()),max(variavel[1].value_counts())) + 7000

            max_y_comparar =  max(max(variavel[0].value_counts()),max(variavel[1].value_counts()))
            min_y_comparar =  max(min(variavel[0].value_counts()),min(variavel[1].value_counts()))


            # Plotar histograma 
            ax1 = fig.add_subplot(((24)/3 ),4,i+1)

            if size <= 20 : # and (max_y_comparar - min_y_comparar) > 5000  :
                size = size
                aux = range(size+1)

                ax1.hist(variavel,color=colors,label=[label_x, label_y], bins=aux, edgecolor='black', linewidth=1.2)
                ax1.set_title(col+ " (Discreto)")
                plt.yscale('log')
                plt.xticks(ticks=np.arange(0.5,size,1), labels=labels)
                plt.tight_layout()
                ax1.legend(fancybox=True, framealpha=0.5, prop={'size': 7})

                for label in  ax1.xaxis.get_ticklabels(): 
                    label.set_rotation(90)

            else:
                size = size
                aux = range(size+1)
                max_eixo_y=  max(max(variavel[0].value_counts()),max(variavel[1].value_counts())) + 1500

                ax1.hist(variavel,color=colors,label=[label_x, label_y], bins='auto', rwidth=3000,edgecolor='black', linewidth=0.8)

                plt.yscale('log')
                ax1.set_title(col + " (Contínuo)") 

                #plt.xticks(ticks=np.arange(0.5,size,1), labels=labels)
                plt.tight_layout()
                ax1.legend(fancybox=True, framealpha=0.5, prop={'size': 7})
                for label in  ax1.xaxis.get_ticklabels(): 
                    label.set_rotation(90)

            